# Data setup

In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/duong/STLF/Data/merge_weather_energy_hanoi_20202025_cleaned.csv', parse_dates=["timestamp"])
df

,Temperature,Weather,Precipitation,Chance of snow,Humidity,Wind,Wind Gust,Wind Degree,Wind Direction,Cloud Cover,Visibility,timestamp,is_weekend,season,is_holiday,total_consumption_mw
0,25.4,Patchy rain possible,0.6,0.0,89.0,2.194444,4.388889,295.0,WNW,89.0,9.0,2020-01-01 00:00:00,0,winter,False,1790.10
1,25.1,Partly cloudy,0.0,0.0,90.0,2.611111,5.111111,297.0,WNW,34.0,10.0,2020-01-01 01:00:00,0,winter,False,1452.26
2,24.7,Patchy rain possible,0.0,0.0,91.0,2.805556,5.500000,309.0,NW,87.0,10.0,2020-01-01 02:00:00,0,winter,False,1483.75
3,24.5,Cloudy,0.0,0.0,92.0,2.611111,4.888889,325.0,NW,71.0,10.0,2020-01-01 03:00:00,0,winter,False,1890.07
4,24.1,Patchy rain possible,0.0,0.0,93.0,2.305556,4.000000,326.0,NNW,100.0,10.0,2020-01-01 04:00:00,0,winter,False,1371.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47444,26.8,Partly cloudy,0.0,0.0,84.0,1.888889,3.388889,109.0,ESE,56.0,10.0,2025-05-30 20:00:00,0,spring,False,2449.28
47445,26.5,Partly cloudy,0.0,0.0,86.0,2.388889,4.194444,126.0,SE,52.0,10.0,2025-05-30 21:00:00,0,spring,False,2554.05
47446,26.1,Patchy rain possible,0.0,0.0,88.0,2.388889,4.194444,149.0,SSE,84.0,10.0,2025-05-30 22:00:00,0,spring,False,1895.41
47447,25.6,Patchy rain possible,0.5,0.0,91.0,2.305556,4.111111,152.0,SSE,76.0,9.0,2025-05-30 23:00:00,0,spring,False,1558.67


In [2]:
CAT_COLS   = ['Weather','Wind Direction','season','is_holiday']
EXOG_NUM   = ['Temperature','Precipitation','Humidity',
              'Wind','Wind Gust','Wind Degree','Cloud Cover','Visibility']
HORIZON    = 24                       # forecast t+1…t+24
TARGET_LAGS   = range(1,49)           # lags of target (1…48)
ROLL_WINDOWS  = [3,6,12,24,48]        # rolling windows
EXOG_LAGS     = [0,1,3,6,12,24]       # lags for exogenous vars
TARGET_COL = 'total_consumption_mw'
TIME_COL   = 'timestamp'

In [3]:
type(df['timestamp'].loc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
df = df.rename(columns={"timestamp": "ds", "total_consumption_mw": "y"})
df

,Temperature,Weather,Precipitation,Chance of snow,Humidity,Wind,Wind Gust,Wind Degree,Wind Direction,Cloud Cover,Visibility,ds,is_weekend,season,is_holiday,y
0,25.4,Patchy rain possible,0.6,0.0,89.0,2.194444,4.388889,295.0,WNW,89.0,9.0,2020-01-01 00:00:00,0,winter,False,1790.10
1,25.1,Partly cloudy,0.0,0.0,90.0,2.611111,5.111111,297.0,WNW,34.0,10.0,2020-01-01 01:00:00,0,winter,False,1452.26
2,24.7,Patchy rain possible,0.0,0.0,91.0,2.805556,5.500000,309.0,NW,87.0,10.0,2020-01-01 02:00:00,0,winter,False,1483.75
3,24.5,Cloudy,0.0,0.0,92.0,2.611111,4.888889,325.0,NW,71.0,10.0,2020-01-01 03:00:00,0,winter,False,1890.07
4,24.1,Patchy rain possible,0.0,0.0,93.0,2.305556,4.000000,326.0,NNW,100.0,10.0,2020-01-01 04:00:00,0,winter,False,1371.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47444,26.8,Partly cloudy,0.0,0.0,84.0,1.888889,3.388889,109.0,ESE,56.0,10.0,2025-05-30 20:00:00,0,spring,False,2449.28
47445,26.5,Partly cloudy,0.0,0.0,86.0,2.388889,4.194444,126.0,SE,52.0,10.0,2025-05-30 21:00:00,0,spring,False,2554.05
47446,26.1,Patchy rain possible,0.0,0.0,88.0,2.388889,4.194444,149.0,SSE,84.0,10.0,2025-05-30 22:00:00,0,spring,False,1895.41
47447,25.6,Patchy rain possible,0.5,0.0,91.0,2.305556,4.111111,152.0,SSE,76.0,9.0,2025-05-30 23:00:00,0,spring,False,1558.67


In [5]:
# make sure categoricals are strings
for col in CAT_COLS:
    df[col] = df[col].astype(str)

In [18]:
for col in CAT_COLS:
    print(df[col].unique())

['Patchy rain possible' 'Partly cloudy' 'Cloudy' 'Light drizzle'
 'Light rain shower' 'Patchy light rain' 'Moderate or heavy rain shower'
 'Overcast' 'Patchy light drizzle']
['WNW' 'NW' 'NNW' 'N' 'NNE' 'W' 'SW' 'SSW' 'S' 'SE' 'ESE' 'SSE']
['winter' 'spring' 'summer' 'autumn']
['False' 'tet' 'national']


In [6]:
# One-hot encode
df = pd.get_dummies(df, columns=CAT_COLS, drop_first=False)
df

,Temperature,Precipitation,Chance of snow,Humidity,Wind,Wind Gust,Wind Degree,Cloud Cover,Visibility,ds,...,Wind Direction_SW,Wind Direction_W,Wind Direction_WNW,season_autumn,season_spring,season_summer,season_winter,is_holiday_False,is_holiday_national,is_holiday_tet
0,25.4,0.6,0.0,89.0,2.194444,4.388889,295.0,89.0,9.0,2020-01-01 00:00:00,...,False,False,True,False,False,False,True,True,False,False
1,25.1,0.0,0.0,90.0,2.611111,5.111111,297.0,34.0,10.0,2020-01-01 01:00:00,...,False,False,True,False,False,False,True,True,False,False
2,24.7,0.0,0.0,91.0,2.805556,5.500000,309.0,87.0,10.0,2020-01-01 02:00:00,...,False,False,False,False,False,False,True,True,False,False
3,24.5,0.0,0.0,92.0,2.611111,4.888889,325.0,71.0,10.0,2020-01-01 03:00:00,...,False,False,False,False,False,False,True,True,False,False
4,24.1,0.0,0.0,93.0,2.305556,4.000000,326.0,100.0,10.0,2020-01-01 04:00:00,...,False,False,False,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47444,26.8,0.0,0.0,84.0,1.888889,3.388889,109.0,56.0,10.0,2025-05-30 20:00:00,...,False,False,False,False,True,False,False,True,False,False
47445,26.5,0.0,0.0,86.0,2.388889,4.194444,126.0,52.0,10.0,2025-05-30 21:00:00,...,False,False,False,False,True,False,False,True,False,False
47446,26.1,0.0,0.0,88.0,2.388889,4.194444,149.0,84.0,10.0,2025-05-30 22:00:00,...,False,False,False,False,True,False,False,True,False,False
47447,25.6,0.5,0.0,91.0,2.305556,4.111111,152.0,76.0,9.0,2025-05-30 23:00:00,...,False,False,False,False,True,False,False,True,False,False


In [7]:
df = df.sort_values("ds").reset_index(drop=True)

In [8]:
reg_cols = [c for c in df.columns if c not in ["ds","y"]]

In [9]:
train_size = int(0.7 * len(df))
val_size   = int(0.15 * len(df))

train = df.iloc[:train_size].copy()
val   = df.iloc[train_size:train_size+val_size].copy()
test  = df.iloc[train_size+val_size:].copy()

In [11]:
print(train.shape)
print(val.shape)
print(test.shape)

(33214, 40)
(7117, 40)
(7118, 40)


In [18]:
N        = df.shape[0]
train_end= int(N - 2*8*7*24)
valid_end= int(N - 8*7*24)

train_last8w = df.iloc[:train_end].copy()
val_last8w = df.iloc[train_end:valid_end].copy()
test_last8w = df.iloc[valid_end:].copy()

print(train_last8w.shape)
print(val_last8w.shape)
print(test_last8w.shape)

(44761, 40)
(1344, 40)
(1344, 40)


In [33]:
# Prophet expects continuous regressors normalized; scale them
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train[reg_cols] = scaler.fit_transform(train[reg_cols])
val[reg_cols]   = scaler.transform(val[reg_cols])
test[reg_cols]  = scaler.transform(test[reg_cols])

In [20]:
# Prophet expects continuous regressors normalized; scale them
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_last8w[reg_cols] = scaler.fit_transform(train_last8w[reg_cols])
val_last8w[reg_cols]   = scaler.transform(val_last8w[reg_cols])
test_last8w[reg_cols]  = scaler.transform(test_last8w[reg_cols])

# Feature engineering

# Model Training

In [24]:
import pandas as pd, numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import timedelta

def metrics_df(y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = (np.abs((y_true - y_pred) / y_true.replace(0, np.nan))).mean() * 100
    return pd.Series({"MAE": mae, "RMSE": rmse, "MAPE_%": mape})

def add_all_regressors(m, reg_cols):
    for c in reg_cols:
        m.add_regressor(c)
    return m


In [25]:
def evaluate_continuous(m, df_slice, reg_cols):
    # Prophet needs the full future frame with regressors for those timestamps
    future = df_slice[["ds"] + reg_cols].copy()
    fcst   = m.predict(future)
    y_pred = fcst["yhat"].values
    y_true = df_slice["y"].values
    return metrics_df(pd.Series(y_true), pd.Series(y_pred))

In [26]:
def day_ahead_grid(df_eval, reg_cols):
    # anchors: start of each day such that next 24 hours are fully inside df_eval
    # assume hourly data
    ds_min = df_eval["ds"].min().replace(minute=0, second=0, microsecond=0)
    ds_max = df_eval["ds"].max()
    anchors = []
    t = ds_min
    while t + timedelta(hours=23) <= ds_max:
        anchors.append(t)
        t += timedelta(days=1)
    return anchors

def evaluate_dayahead_24h(m, df_eval, reg_cols):
    records = []
    idx = df_eval.set_index("ds")
    for t0 in day_ahead_grid(df_eval, reg_cols):
        horizon = pd.date_range(t0, periods=24, freq="H")
        # skip if any horizon ts missing
        if not set(horizon).issubset(idx.index):
            continue
        future = idx.loc[horizon, reg_cols].reset_index().rename(columns={"index":"ds"})
        future = future[["ds"] + reg_cols]  # (ds + regressors for the next 24h)
        fcst = m.predict(future)
        y_pred = fcst["yhat"].values
        y_true = idx.loc[horizon, "y"].values
        for h, (yt, yp) in enumerate(zip(y_true, y_pred), start=1):
            records.append({"anchor": t0, "h": h, "y_true": yt, "y_pred": yp})
    out = pd.DataFrame(records)
    # overall day-ahead performance
    overall = metrics_df(out["y_true"], out["y_pred"])
    # per-horizon metrics
    per_h = (
        out.groupby("h")
           .apply(lambda g: metrics_df(g["y_true"], g["y_pred"]))
           .reset_index()
    )
    return overall, per_h, out

### 70-15-15 split

In [35]:
m = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
m = add_all_regressors(m, reg_cols)
m.fit(train[["ds","y"] + reg_cols])

DEBUG:cmdstanpy:input tempfile: /tmp/tmpix7sfsga/m0vor_58.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpix7sfsga/f5_vfmik.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=74406', 'data', 'file=/tmp/tmpix7sfsga/m0vor_58.json', 'init=/tmp/tmpix7sfsga/f5_vfmik.json', 'output', 'file=/tmp/tmpix7sfsga/prophet_model7cwxnmuy/prophet_model-20250923214829.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:48:29 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:48:45 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [36]:
print("70–15–15: VAL (continuous)")
print(evaluate_continuous(m, val, reg_cols))  # if you trained m on train_last8w

print("70–15–15: TEST (continuous)")
print(evaluate_continuous(m, test, reg_cols))


70–15–15: VAL (continuous)
MAE       526.949795
RMSE      692.652180
MAPE_%     24.203186
dtype: float64
70–15–15: TEST (continuous)
MAE       611.472653
RMSE      814.959755
MAPE_%     29.318380
dtype: float64


In [37]:
# Example: evaluate on the last-8-weeks TEST window
overall_da, per_h_da, raw_da = evaluate_dayahead_24h(m, test[["ds","y"] + reg_cols], reg_cols)
print("Day-ahead (next 24h) OVERALL on TEST:\n", overall_da)
print("\nPer-horizon RMSE/MAPE (h = 1..24):")
print(per_h_da[["h","RMSE","MAPE_%"]].to_string(index=False))


/tmp/ipython-input-2524072095.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-2524072095.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-2524072095.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-2524072095.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-2524072095.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-2524072095.py:17: FutureWarning: 'H' is deprec

Day-ahead (next 24h) OVERALL on TEST:
 MAE       610.262598
RMSE      813.626625
MAPE_%     29.294539
dtype: float64

Per-horizon RMSE/MAPE (h = 1..24):
 h        RMSE    MAPE_%
 1  784.554271 26.170781
 2  875.375362 24.515444
 3  963.050162 24.436898
 4  891.946847 25.440653
 5  762.751935 23.783246
 6  522.883190 19.939593
 7  423.549514 12.812361
 8  568.456030 20.384806
 9  702.346968 21.506329
10  633.917754 18.085320
11  583.765801 15.992622
12  610.880926 20.804362
13  542.146570 22.253275
14  633.256999 30.776561
15  562.322316 30.509289
16  734.504589 43.757391
17  526.269634 31.948225
18  623.739929 34.871160
19  480.275120 24.092099
20  422.480532 18.670384
21  471.292431 20.154428
22 1318.849535 60.231867
23 1891.858041 79.065001
24 1284.378603 52.866846


/tmp/ipython-input-2524072095.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-2524072095.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: metrics_df(g["y_true"], g["y_pred"]))


### last 8 weeks as test set split

In [22]:
m = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
m = add_all_regressors(m, reg_cols)
m.fit(train_last8w[["ds","y"] + reg_cols])

DEBUG:cmdstanpy:input tempfile: /tmp/tmph38jtrl4/g56elim9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmph38jtrl4/q7dki0h1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97824', 'data', 'file=/tmp/tmph38jtrl4/g56elim9.json', 'init=/tmp/tmph38jtrl4/q7dki0h1.json', 'output', 'file=/tmp/tmph38jtrl4/prophet_model_i19qvy1/prophet_model-20250924085324.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:53:24 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:54:08 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [28]:
print("Last 8 weeks split: VAL (continuous)")
print(evaluate_continuous(m, val_last8w, reg_cols))  # if you trained m on train_last8w

print("Last 8 weeks split: TEST (continuous)")
print(evaluate_continuous(m, test_last8w, reg_cols))

Last 8 weeks split: VAL (continuous)
MAE       445.538646
RMSE      583.318472
MAPE_%     22.234466
dtype: float64
Last 8 weeks split: TEST (continuous)
MAE        812.588113
RMSE      1046.311186
MAPE_%      31.155439
dtype: float64


In [27]:
# Example: evaluate on the last-8-weeks TEST window
overall_da, per_h_da, raw_da = evaluate_dayahead_24h(m, test_last8w[["ds","y"] + reg_cols], reg_cols)
print("Day-ahead (next 24h) OVERALL on TEST:\n", overall_da)
print("\nPer-horizon RMSE/MAPE (h = 1..24):")
print(per_h_da[["h","RMSE","MAPE_%"]].to_string(index=False))

/tmp/ipython-input-13223931.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-13223931.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-13223931.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-13223931.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-13223931.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-13223931.py:17: FutureWarning: 'H' is deprecated and wil

Day-ahead (next 24h) OVERALL on TEST:
 MAE        812.588113
RMSE      1046.311186
MAPE_%      31.155439
dtype: float64

Per-horizon RMSE/MAPE (h = 1..24):
 h        RMSE    MAPE_%
 1  534.293637 37.492719
 2  214.647117 15.118552
 3  268.364513 15.600799
 4  701.082143 50.193957
 5  236.984185 13.689725
 6  345.781300 16.676096
 7  292.642579 10.480661
 8  622.270808 17.925404
 9  966.214337 24.255448
10 1793.575352 46.758980
11 1941.682280 46.545616
12 1905.723062 42.995370
13 1878.909525 44.268589
14 1488.576204 38.442985
15 1156.315067 30.398091
16  615.161121 17.948740
17  469.278828 10.626799
18  583.206555 21.296886
19  469.035330 15.245062
20  971.539261 33.849610
21  967.527973 34.966823
22 1301.370574 62.460921
23  980.730901 56.687910
24  633.213780 43.804792


/tmp/ipython-input-13223931.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(t0, periods=24, freq="H")
/tmp/ipython-input-13223931.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: metrics_df(g["y_true"], g["y_pred"]))
